In [3]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define the directory containing the PDF files
pdf_directory = './data'

논문을 벡터 db에 넣기

In [4]:
# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************itN3. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

### 인적정보 가져오기

In [12]:
import json

class User:
    def __init__(self, info_dir):
        # Read the JSON file
        with open(info_dir, 'r') as file:
            data = json.load(file)
        self.name = data['personal_information']['name']
        self.phone = data['personal_information']['phone']
        self.age = data['personal_information']['age']
        self.gender = data['personal_information']['gender']
        self.merry = data['personal_information']['merry']
        self.children = data['personal_information']['children']
        self.religion = data['personal_information']['religion']
        self.economy_states = data['personal_information']['economy_states']
        self.health_states = data['personal_information']['health_states']
        self.living_arrangement = data['personal_information']['living_arrangement']
# Specify the path to the JSON file
info_dir = '/Users/seungwoo/Workspace/IPACT/data/ipact_personal_main/data2.json'
user = User(info_dir)
print(user.religion)

0


### 프롬프트

In [15]:
import json
class QA:
    def __init__(self,file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            self.data = json.load(file)
    def process(self):
        self.data = self.data['subjective_questions']
        return self.data

qa = QA('./data/ipact_personal_main/data2.json')
print(qa.process())
class KGLSData:
    def __init__(self, file_path):
        # JSON 파일 읽기
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        self.kgls = data['KGLS']
        
    def get_question(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['question']
        return None
    
    def get_user_choice(self, q_num):
        for item in self.kgls:
            if item['q_num'] == q_num:
                return item['user_choose']
        return None
    
    def get_all_questions(self):
        return [item['question'] for item in self.kgls]
    def get_all_user_choices(self):
        return [item['user_choose'] for item in self.kgls]
# 파일 경로 지정
file_path = './data/kgls_dummy.json'
kgls_data = KGLSData(file_path)

# 예시 출력
print(kgls_data.get_question(1))       # "나는 가족들과 매일 대화를 나눈다."
print(kgls_data.get_user_choice(1))    # 3
print(kgls_data.get_all_questions())   # 모든 질문 리스트 출력

with open("data/prompt_output.txt", "r") as f:
    KGLS = f.read()
personal_info="""
#설문지 인적 정보 양식
본 설문지의 인적 정보의 답변 값에 대한 숫자와 값을 일치시켰습니다. 괄호 안의 값을 참고하세요
이름(name) : 00
연령(age) : 0(65-74세) / 1(75-84세) / 2(85세 이상)
성별(gender) : male / female
결혼 상태(merry) : 0(미혼) / 1(기혼) / 2(이혼/별거) / 3(사별)
거주 형태(living_arrangement) : 0(혼자 거주) / 1(동거)
자녀 수 (chlidren) : 0(없음) / 2(1-2명) / 3(3명 이상)
종교(religion): 1(있음) / 0(없음)
지각된 경제 상태(economy_states): 0(나쁨) / 1(보통) / 2(좋음)
지각된 건강 상태(health_states): 0(나쁨) / 1(보통) / 2(좋음)
"""
SYS_PROMPT = f"""
    사용자의 한국형 외로움 분석과 인공지능 대화 상대가 되기위한 시스템 프롬프트 설정을 위한 시스템입니다. 아래의 정보와 입력된 정보를 활용하세요. 반드시 아래의 정보와 문맥을 참고하세요.\\
    1. 사용자의 인적정보
    2. KGLS 질문과 사용자의 답변의 분석결과
    3. 외로움 파악을 위한 주관식 질문사용자가 입력한 답변
    4. 노인들의 외로움과 한국형 외로움에 대한 연구
    
    {personal_info}
    사용자의 인적정보 = 이름: {user.name},
        나이: {user.age}, 
        성별: {user.gender}, 
        결혼 여부: {user.merry}, 
        자녀 수: {user.children}, 
        종교: {user.religion}, 
        거주 형태: {user.living_arrangement},
        인지된 경제 상태: {user.economy_states}, 
        인지된 건강 상태: {user.health_states},
        
        \\
    KGLS 질문과 사용자의 답변의 분석결과 = {KGLS} \\

    
    
"""
INPUT_PROMPT = f"""

"""

{'loneliness_situations': '아픈데 혼자 식사를 할 때', 'realization_of_loneliness': '누워서 생각하면 이 생각 저 생각으로 외로움을 느낌', 'needs_during_loneliness': '주변인들과 왕래가 잦아야하고,회관에서 하는 프로그램이 많았으면 함'}
나는 가족들과 매일 대화를 나눈다.
3
['나는 가족들과 매일 대화를 나눈다.', '나는 가깝게 지내는 사람들이 있다.', '나는 쓸모없는 사람이라고 느껴진다.', '나를 이해해주는 사람이 있다.', '사람들은 나와 겉으로만 어울리는 것 같다.', '나는 자녀에게 고민을 이야기할 수 있다.', '나는 주변 사랑과의 관계가 만족스럽다.', '가족들은 나를 예전처럼 대해주지 않는다.', '친구나 이웃들은 나에게 관심을 둔다.', '내가 아플 때 나를 보살펴줄 가족이 있다.', '나는 의지할 친구가 있다.', '나는 온종일 할 일없이 시간을 보낸다.', '나는 사회에서 필요한 사람이다.', '나는 가족에게 의지할 수 있다.']


모델 선언

In [17]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''
    3. 외로움 파악을 위한 주관식 질문과 사용자가 입력한 답변 = {question} \\
    4. 노인들의 외로움과 한국형 외로움에 대한 연구 = {context}\\
    
    출력은 앞서 입력된 내용을 바탕으로 출력해야합니다. 특히 분석결과는 3,4번 항목을 참고하여 충분히 설명해야하며, 한국형 외로움의 질적연구에서 주관적 답변을 분석했던 결과를 바탕으로 사용자의 외로움을 예측하고 해당하는 외로움들을 출력해야합니다.\\
    출력 예시:
    [외로움 종류, 분석 결과]
    - 000 님이 해당하는 한국형 외로움은 OOO 00%, OOO 00% ...입니다. 
    - OOO 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
    - OOO 외로움은 ..., ...한 특징을 가지고 있으며, 사용자의 경우 ...한 경험으로 인해 나타납니다..\\
    - ...
    [판단 근거]
    - 외로움 종류에 대한 설명
    - 외로움에 관한 연구논문에서 찾은 판단 근거
    
    [대화 시스템 프롬프트]
    - 당신은 000님을 위한 대화 상대가 되어주어야합니다. 아래의 참고사항을 반영해 출력하세요.
    - 대화중 사용자의 기억은 반드시 기억해야합니다.
    - 말투는 ...하고, ...해야할 것입니다. 
    - ...
'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(api_key=OPENAI_API_KEY,model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.

input_message = str(qa) + INPUT_PROMPT  # 추가적인 입력 프롬프트가 이어집니다.
text = rag_chain.invoke(input_message)
# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
print(text)


[외로움 종류, 분석 결과]
- 이경자 님이 해당하는 한국형 외로움은 "중간 정도의 외로움" 75%, "외부관계에서 오는 외로움" 60%, "혼자라고 느껴지는 쓸쓸함" 50%입니다.
- "중간 정도의 외로움"은 가족과 떨어져 지내는 경험, 건강 상태의 악화, 경제적 어려움 등으로 인해 나타납니다. 이경자 님의 경우, 이혼 후 혼자 지내는 시간이 많아지면서 이러한 외로움을 느끼고 있습니다.
- "외부관계에서 오는 외로움"은 가족, 친구, 사회적 관계의 부족으로 인해 느끼는 외로움입니다. 이경자 님은 자녀들이 성인이 되어 독립한 후, 가족과의 관계에서 외로움을 느끼며, 특히 혼자 남아있을 때 외로움을 크게 느낍니다.
- "혼자라고 느껴지는 쓸쓸함"은 배우자와의 사별, 가족과의 소원한 관계 등으로 인해 느끼는 외로움입니다. 이경자 님은 이혼 후 혼자 지내는 시간이 많아지면서 이러한 쓸쓸함을 느끼고 있습니다.

[판단 근거]
- 외로움 종류에 대한 설명:
  1. "중간 정도의 외로움"은 KGLS 점수 29-42점에 해당하며, 이는 가족과의 분리, 건강 문제, 경제적 어려움 등 다양한 요인으로 인해 발생합니다.
  2. "외부관계에서 오는 외로움"은 가족, 친구, 사회적 관계의 부족으로 인해 느끼는 외로움으로, 연구에 따르면 노인들은 배우자와의 사별이나 가족과의 관계 소홀, 무관심으로 인해 외로움을 경험합니다.
  3. "혼자라고 느껴지는 쓸쓸함"은 배우자와의 사별, 가족과의 소원한 관계 등으로 인해 느끼는 외로움으로, 연구에 따르면 노인들은 배우자와의 사별로 인해 혼자 남겨진 느낌을 받으며, 견디기 힘든 외로움을 경험합니다.

- 외로움에 관한 연구논문에서 찾은 판단 근거:
  1. "중간 정도의 외로움"은 KGLS 점수 42점으로, 이는 중간 정도의 외로움에 해당합니다.
  2. "외부관계에서 오는 외로움"은 연구에서 노인들이 가족, 친구나 사회적 관계의 부족으로 인해 느끼는 외로움으로 구성되며, 이경자 님의 경우 자녀들이 독립한 후 가족과의 관계에서 외로움을 느

In [18]:
with open('data/prompt_output.txt', 'w') as file:
    file.write(text)